<a href="https://colab.research.google.com/github/dougj892/Jupyter-notebooks/blob/master/RStan_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
install.packages("rstan")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘checkmate’, ‘matrixStats’, ‘StanHeaders’, ‘inline’, ‘gridExtra’, ‘loo’, ‘RcppEigen’




In [3]:
library(rstan)

Loading required package: StanHeaders

Loading required package: ggplot2

rstan (Version 2.19.2, GitRev: 2e1f913d3ca3)

For execution on a local, multicore CPU with excess RAM we recommend calling
options(mc.cores = parallel::detectCores()).
To avoid recompilation of unchanged Stan programs, we recommend calling
rstan_options(auto_write = TRUE)



In [4]:
pkgbuild::has_build_tools(debug = TRUE)

Trying to compile a simple C file



Running /usr/lib/R/bin/R CMD SHLIB foo.c
gcc -std=gnu99 -I"/usr/share/R/include" -DNDEBUG     -fpic  -g -O2 -fdebug-prefix-map=/build/r-base-t3diwe/r-base-3.6.2=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -g  -c foo.c -o foo.o
gcc -std=gnu99 -shared -L/usr/lib/R/lib -Wl,-Bsymbolic-functions -Wl,-z,relro -o foo.so foo.o -L/usr/lib/R/lib -lR


[1] TRUE

In [0]:
stan_code <- "data {
  int<lower=0> J;         // number of schools 
  real y[J];              // estimated treatment effects
  real<lower=0> sigma[J]; // standard error of effect estimates 
}
parameters {
  real mu;                // population treatment effect
  real<lower=0> tau;      // standard deviation in treatment effects
  vector[J] eta;          // unscaled deviation from mu by school
}
transformed parameters {
  vector[J] theta = mu + tau * eta;        // school treatment effects
}
model {
  target += normal_lpdf(eta | 0, 1);       // prior log-density
  target += normal_lpdf(y | theta, sigma); // log-likelihood
}"

In [0]:
schools_dat <- list(J = 8, 
                    y = c(28,  8, -3,  7, -1,  1, 18, 12),
                    sigma = c(15, 10, 16, 11,  9, 11, 10, 18))

In [10]:
fit <- stan(model_code = stan_code, data = schools_dat)


SAMPLING FOR MODEL 'b02283c080bb32a2f56ccbcb1d8afc68' NOW (CHAIN 1).
Chain 1: 
Chain 1: Gradient evaluation took 1.4e-05 seconds
Chain 1: 1000 transitions using 10 leapfrog steps per transition would take 0.14 seconds.
Chain 1: Adjust your expectations accordingly!
Chain 1: 
Chain 1: 
Chain 1: Iteration:    1 / 2000 [  0%]  (Warmup)
Chain 1: Iteration:  200 / 2000 [ 10%]  (Warmup)
Chain 1: Iteration:  400 / 2000 [ 20%]  (Warmup)
Chain 1: Iteration:  600 / 2000 [ 30%]  (Warmup)
Chain 1: Iteration:  800 / 2000 [ 40%]  (Warmup)
Chain 1: Iteration: 1000 / 2000 [ 50%]  (Warmup)
Chain 1: Iteration: 1001 / 2000 [ 50%]  (Sampling)
Chain 1: Iteration: 1200 / 2000 [ 60%]  (Sampling)
Chain 1: Iteration: 1400 / 2000 [ 70%]  (Sampling)
Chain 1: Iteration: 1600 / 2000 [ 80%]  (Sampling)
Chain 1: Iteration: 1800 / 2000 [ 90%]  (Sampling)
Chain 1: Iteration: 2000 / 2000 [100%]  (Sampling)
Chain 1: 
Chain 1:  Elapsed Time: 0.039065 seconds (Warm-up)
Chain 1:                0.034783 seconds (Sampling)


Warning message:
“There were 1 divergent transitions after warmup. Increasing adapt_delta above 0.8 may help. See
http://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup”
Warning message:
“Examine the pairs() plot to diagnose sampling problems
”


In [11]:
print(fit)

Inference for Stan model: b02283c080bb32a2f56ccbcb1d8afc68.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

           mean se_mean   sd   2.5%    25%    50%    75%  97.5% n_eff Rhat
mu         7.82    0.11 4.98  -1.65   4.49   7.81  11.00  17.59  2125 1.00
tau        6.54    0.15 5.57   0.18   2.42   5.21   9.19  20.34  1423 1.00
eta[1]     0.40    0.02 0.94  -1.53  -0.19   0.42   1.05   2.19  3548 1.00
eta[2]     0.01    0.01 0.87  -1.72  -0.59   0.00   0.60   1.73  3924 1.00
eta[3]    -0.18    0.01 0.91  -1.89  -0.81  -0.18   0.42   1.66  4010 1.00
eta[4]    -0.02    0.01 0.87  -1.71  -0.60  -0.03   0.55   1.69  3944 1.00
eta[5]    -0.37    0.02 0.88  -2.02  -0.95  -0.39   0.17   1.39  3338 1.00
eta[6]    -0.20    0.01 0.89  -1.97  -0.77  -0.20   0.37   1.55  3663 1.00
eta[7]     0.34    0.01 0.89  -1.43  -0.25   0.36   0.94   2.07  3564 1.00
eta[8]     0.06    0.01 0.95  -1.84  -0.58   0.07   0.71   1.94  4241 1.